In [12]:
# Libraries
import os
import pandas as pd
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv

In [13]:
# Step 1: Load environment variables from pgadmin.env file
load_dotenv('database_info.env')

# Step 2: Read environment variables
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

In [14]:
# Step 3: Load the CSV file into a pandas DataFrame
csv_file = 'data/social_media_entertainment_data.csv'  # Replace with the path to your CSV file
df = pd.read_csv(csv_file)

In [15]:
# Step 4: Connect to PostgreSQL
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    cursor = conn.cursor()
    print("Connected to PostgreSQL successfully!")
except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL: {e}")
    exit()

Connected to PostgreSQL successfully!


In [16]:
# Step 5: Generate the CREATE TABLE SQL statement
table_name = 'kaggle_data'  # Replace with your desired table name
columns = []

# Infer data types from the CSV file
for col in df.columns:
    # Get the first non-null value in the column
    sample_value = df[col].dropna().iloc[0] if not df[col].dropna().empty else None

    # Map Python data types to PostgreSQL data types
    if isinstance(sample_value, str):
        sql_type = 'TEXT'
    elif isinstance(sample_value, int):
        sql_type = 'INTEGER'
    elif isinstance(sample_value, float):
        sql_type = 'FLOAT'
    else:
        sql_type = 'TEXT'  # Default to TEXT if unsure

    # Add the column definition to the list
    columns.append(f'"{col}" {sql_type}')  # Use double quotes for column names

# Combine columns into a CREATE TABLE SQL statement
create_table_sql = f"CREATE TABLE {table_name} ({', '.join(columns)});"

# Step 6: Execute the CREATE TABLE statement
try:
    cursor.execute(create_table_sql)
    print(f"Table '{table_name}' created successfully.")
except psycopg2.Error as e:
    print(f"Error creating table: {e}")

# Step 7: Import the CSV data into the table
try:
    # Save the DataFrame to a temporary CSV file (required for COPY)
    temp_csv = 'temp.csv'
    df.to_csv(temp_csv, index=False, header=False)

    # Use the COPY command to import the data
    with open(temp_csv, 'r') as f:
        cursor.copy_expert(f"COPY {table_name} FROM STDIN WITH CSV", f)
    print(f"Data imported into '{table_name}' successfully.")
except psycopg2.Error as e:
    print(f"Error importing data: {e}")
finally:
    # Clean up the temporary CSV file
    import os
    if os.path.exists(temp_csv):
        os.remove(temp_csv)

# Step 8: Commit and close the connection
conn.commit()
cursor.close()
conn.close()
print("Connection closed.")


Error creating table: relation "kaggle_data" already exists

Error importing data: current transaction is aborted, commands ignored until end of transaction block

Connection closed.
